# 🏃 Strava Analytics Dashboard

---

### 1. Initialization
We start by loading our custom `analyst` module. This module contains all the logic for our Smart Commands. 

In [35]:
%load_ext autoreload
%autoreload 2
import src.analyst as strava
import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# --- FIRST TIME SETUP: Authentication ---
# Run this cell ONLY if you need to authenticate with Strava for the first time or re-authenticate.
from src.auth import StravaAuth
auth = StravaAuth()
auth.authenticate()


=== Strava Authentication ===

Opening browser for authentication...
If the browser doesn't open, visit this URL manually:

https://www.strava.com/oauth/authorize?client_id=188749&redirect_uri=http%3A%2F%2Flocalhost%3A8501&response_type=code&scope=read%2Cactivity%3Aread_all&approval_prompt=auto

After authorizing, you'll be redirected to a URL like:
http://localhost:8501/?state=&code=XXXXX&scope=read,activity:read_all

Copy the ENTIRE URL and paste it here:

Authorization code received: e0f52164d1...
Exchanging code for access token...
Token saved to cache/strava_token.json

Authentication successful!
Access token expires at: Wed Jan 21 22:43:31 2026


{'token_type': 'Bearer',
 'expires_at': 1769031811,
 'expires_in': 21600,
 'refresh_token': 'b9964dba736348a829bddadb9e1a0776eae47651',
 'access_token': '5d9256d044e862596bcba9078837a6414b8a4676',
 'athlete': {'id': 37948241,
  'username': 'ssiddhanth',
  'resource_state': 2,
  'firstname': 'Siddhanth',
  'lastname': 'Sharma',
  'bio': None,
  'city': None,
  'state': None,
  'country': None,
  'sex': 'M',
  'premium': False,
  'summit': False,
  'created_at': '2019-01-07T04:25:56Z',
  'updated_at': '2025-11-14T03:08:35Z',
  'badge_type_id': 0,
  'weight': None,
  'profile_medium': 'https://graph.facebook.com/2338387939529330/picture?height=256&width=256',
  'profile': 'https://graph.facebook.com/2338387939529330/picture?height=256&width=256',
  'friend': None,
  'follower': None}}

In [9]:
# --- FIRST TIME SETUP: Fetch Data ---
# Run this cell ONLY to download your data for the first time.
# For daily updates, use strava.refresh() instead.
from src.data_manager import DataManager
dm = DataManager()
dm.fetch_and_cache_activities(force_refresh=True)

Fetching activities from Strava API...
Fetching activities from Strava API...
Fetched page 1: 98 activities (Total: 98)
Total activities fetched: 98
Activities saved to data/activities.json
Activities saved to data/activities.csv


[{'resource_state': 2,
  'athlete': {'id': 37948241, 'resource_state': 1},
  'name': 'Afternoon Walk',
  'distance': 1089.5,
  'moving_time': 796,
  'elapsed_time': 914,
  'total_elevation_gain': 3.6,
  'type': 'Walk',
  'sport_type': 'Walk',
  'device_name': 'Apple Watch Series 8',
  'id': 17104244109,
  'start_date': '2026-01-19T14:52:27Z',
  'start_date_local': '2026-01-19T15:52:27Z',
  'timezone': '(GMT+01:00) Europe/Berlin',
  'utc_offset': 3600.0,
  'location_city': None,
  'location_state': None,
  'location_country': None,
  'achievement_count': 0,
  'kudos_count': 0,
  'comment_count': 0,
  'athlete_count': 1,
  'photo_count': 0,
  'map': {'id': 'a17104244109',
   'summary_polyline': 'mlvdIsk{j@THDSDYzAsBVcBWwDK{BoAcFAOH}CDS_BaFs@iB}@eFk@i@W}@Ey@C_EEgB[i@m@SM_BOHQ?ACAc@EWOUME',
   'resource_state': 2},
  'trainer': False,
  'commute': False,
  'manual': False,
  'private': False,
  'visibility': 'everyone',
  'flagged': False,
  'gear_id': None,
  'start_latlng': [53.367912, 7

In [10]:
strava.refresh()

🔄 Refreshing data from Strava...
Fetching activities from Strava API...
Fetching activities from Strava API...
Fetched page 1: 98 activities (Total: 98)
Total activities fetched: 98
Activities saved to data/activities.json
Activities saved to data/activities.csv
Fetching athlete info from API...
Athlete info cached to data/athlete_info.json
Loading activities from data/activities.csv
✅ Data refreshed successfully!


### 2. Global Overview
First, let's get a high-level picture of the athlete's performance. 
The `show("summary")` command calculates total distance, elevation, and average speed across all historical activities.

In [11]:
strava.show("summary")

Loading athlete info from cache...
--- Global Statistics for Siddhanth ---


Metric,Value
Total Activities,98
Total Distance,239.5 km
Total Elevation,556 m
Moving Time,22.5 hrs
Avg Speed,15.2 km/h
Max Speed,41.4 km/h
Min Speed,4.1 km/h


Now, let's see the breakdown of different sports. 
The `show("types")` command generates a Donut Chart visualization to show the distribution of Rides vs. Walks vs. Runs.

In [12]:
strava.show("types")

### 3. Comparison (This Month vs Last Month)
A key question for any athlete is "Am I improving?"
The `compare("month")` command automatically fetches data for the current month and the previous month, comparing them side-by-side across Distance, Elevation, and Activity Count.

In [13]:
strava.compare("month")

### 4. Recent Activities
Let's look at the raw data for our most recent efforts. 
The `show("recent")` command displays a table of the latest activities, which is useful for checking recent consistency or identifying specific activity IDs.

In [14]:
# Show the latest 10 activities
strava.show("recent", limit=5)

,start_date_local,name,type,distance,total_elevation_gain
0,2026-01-19 15:52:27+00:00,Afternoon Walk,Walk,1089.5,3.6
1,2026-01-16 16:55:08+00:00,Afternoon Ride,Ride,1171.9,4.4
2,2026-01-16 12:07:01+00:00,Lunch Walk,Walk,1407.3,5.4
3,2026-01-11 18:22:05+00:00,Evening Walk,Walk,2186.8,13.6
4,2026-01-10 21:32:32+00:00,Night Walk,Walk,2532.1,19.0


### 5. Interactive Mapping

We can drill down into specific activities. 
The `plot("map")` command uses the Google Polyline algorithm to decode the GPS data of an activity and renders an interactive Folium map.

*Arguments:* `index=0` selects the most recent activity.

In [16]:
# Index 0 = Latest Activity
strava.details(29)
strava.plot("map", index=29)

Loading athlete info from cache...
--- Activity Details: Morning Ride ---
Workout type: Ride
Date:       2025-11-04
Time:       09:28:39 - 09:42:05
Duration:   0h 13m
Distance:   3.93 km
Calories:   118 kcal (est.)
Avg Pace:   3.42 min/km
Avg HR:     164.6 bpm
Displaying map for: Morning Ride (2025-11-04)


### 6. Smart Filtering (The "Drill Down")

Dashboards often become cluttered with mixed data (e.g., mixing Cycle speeds with Walking speeds). 
The `filter()` command sets a global context for the analyst. Here, we restrict all subsequent analysis to only **Walks**.

In [27]:
# Apply a global filter for 'Walk'
strava.filter(sport="Walk")
strava.show("summary")

✅ Filter applied: Activity Type = 'Walk' (32 activities)
Loading athlete info from cache...
--- Global Statistics for Siddhanth ---


Metric,Value
Total Activities,32
Total Distance,61.9 km
Total Elevation,256 m
Moving Time,12.9 hrs
Avg Speed,4.9 km/h
Max Speed,6.7 km/h
Min Speed,4.1 km/h


Let's verify the filter worked. 
Running `show("summary")` again should now only reflect statistics for our Walks.

In [18]:
# Verify filter works - Summary should now only show Walk stats
strava.filter(sport="Ride")
strava.show("summary")

✅ Filter applied: Activity Type = 'Ride' (66 activities)
Loading athlete info from cache...
--- Global Statistics for Siddhanth ---


Metric,Value
Total Activities,66
Total Distance,177.6 km
Total Elevation,300 m
Moving Time,9.6 hrs
Avg Speed,20.2 km/h
Max Speed,41.4 km/h
Min Speed,10.7 km/h


### 7. Trend Analysis

Now that we have filtered for 'Walk', we can analyze specific trends.
The `plot("trend")` command creates a time-series chart. We use the `metric="pace"` argument to visualize our walking pace (min/km) over time.

In [29]:
# 'pace' is a smart alias for 'minutes per km'
strava.filter(reset=True)
strava.plot("trend", metric="pace") ## Change the graph heading to Minutes per KM
strava.plot("trend",metric ="speed")



✅ Filters cleared. Using all data.


### 8. Heatmap

To understand our activity habits, we use a heatmap.
The `plot("heatmap")` command aggregates our data by **Month** vs. **Day of Week**, showing us exactly when we are most active.

In [20]:
strava.plot("heatmap", metric="distance_km")

### 9. Reset

Finally, we can clear our filters to return to the full dataset using `filter(reset=True)`.

In [21]:
strava.filter(reset=True)

✅ Filters cleared. Using all data.


## Additional Feature

### 7. AI Assistant (Ollama)
We've integrated a local LLM (Ollama) to answer questions about your data.
Ensure `ollama serve` is running in your terminal.

In [22]:
strava.ask("Give me a summary of my performance")

🤖 Asking Ollama (mistral-nemo:latest)...


"You've completed an impressive 98 activities, covering 239.5 km and climbing 556 meters! Your longest activity was a 5.2 km ride. Keep up the great work!"

In [31]:
strava.ask("Which my fastest and lowest speed while riding and while walking? Give me in a table")

🤖 Asking Ollama (mistral-nemo:latest)...


| Activity Type | Fastest Speed (km/h) | Slowest Speed (km/h) |
| --- | --- | --- |
| Ride | 28.9 | 10.3 |
| Walk | 7.5 | 4.2 |

Keep up the great work! You're making progress in both speed and distance.

In [34]:
strava.ask("I am planning to do a 5k marathon run in 2026. How can I get started? Looking at the data do you think I can do it? Give me a weekly plan till my run day on March 1")

🤖 Asking Ollama (mistral-nemo:latest)...


Absolutely, you're well on your way! You've already completed activities up to 5.2 km. Here's a weekly plan to help you prepare for your 5k marathon:

**Weeks 1-3 (Feb 6 - Feb 20):**
- Continue with your walks and add one longer walk per week, aiming for 4-5 km.
- Ensure you have at least one rest day each week.

**Week 4 (Feb 21 - Feb 27):**
- Introduce a short run-walk routine: run for 30 seconds, then walk for 90 seconds. Repeat for 20 minutes total.
- Maintain your longer walk on another day of the week.

**Weeks 5-6 (Feb 28 - Mar 13):**
- Gradually increase your running intervals while decreasing walking intervals:
  - Week 5: Run 45 sec, Walk 75 sec
  - Week 6: Run 1 min 30 sec, Walk 90 sec
- Ensure you have at least one rest day each week.

**Week 7 (Mar 14 - Mar 20):**
- Try to run for 5k without stopping. If you need breaks, that's okay!
- Rest on March 18 and 19 to prepare for your race day on March 20.

You're doing great! Stay consistent, listen to your body, and keep up the fantastic work. You've got this! 🏃‍♂️💪

In [ ]:
# Additional:
# strava.ask("What all types of exercise I have done?")
# strava.ask("What should I do to improve based on my recent activity?")
# strava.ask("Give me a summary of all the workouts i have done in November 2025, categorise based on Ride and Walk, what aspects can I improve?")